In [15]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font

# Функция для получения данных о вакансиях
def get_all_vacancies(keyword, days_ago=7):
    url = "https://api.hh.ru/vacancies"
    vacancies = []
    
    # Определяем дату для фильтрации вакансий за последнюю неделю
    date_from = (datetime.now() - timedelta(days=days_ago)).strftime('%Y-%m-%d')
    
    page = 0
    while True:
        params = {
            'text': keyword,
            'search_field': 'name',
            'schedule': 'remote',  # фильтр удаленной работы
            'per_page': 100,  # Максимально допустимое количество на страницу
            'page': page,
            'date_from': date_from
        }
        
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            if not data['items']:
                break  # Прекращаем сбор, если вакансий больше нет

            for item in data['items']:
                # Извлекаем дополнительные данные о вакансии
                employer_name = item['employer']['name'] if item.get('employer') else 'Не указано'
                employment_type = item['employment']['name'] if item.get('employment') else 'Не указано'
                
                # Обрабатываем теги вакансии
                specializations = ', '.join([spec['name'] for spec in item.get('specializations', [])])
                professional_roles = ', '.join([role['name'] for role in item.get('professional_roles', [])])
                
                # Извлекаем данные об опыте работы
                experience = item['experience']['name'] if item.get('experience') else 'Не указано'
                
                # Создаем запись о вакансии
                vacancy = {
                    'Название': item['name'],
                    'Работодатель': employer_name,
                    'Дата публикации': item['published_at'][:10],
                    'Зарплата': parse_salary(item['salary']),
                    'Город': item['area']['name'],
                    'Тип занятости': employment_type,
                    'Теги вакансии': f"{specializations}, {professional_roles}",
                    'Опыт работы': experience,
                    'Ссылка': item['alternate_url']
                }
                vacancies.append(vacancy)
            
            page += 1  # Переходим к следующей странице
        else:
            print(f"Ошибка: {response.status_code}")
            break
    
    return vacancies

# Функция для обработки данных о зарплате
def parse_salary(salary):
    if salary:
        from_salary = salary['from'] if salary['from'] else 'Не указана'
        to_salary = salary['to'] if salary['to'] else 'Не указана'
        return f"{from_salary} - {to_salary} {salary['currency']}"
    return "Не указана"

# Ключевые слова для поиска вакансий
keywords = ["data scientist", "data analyst", "аналитик данных", "BI аналитик", "аналитик"]

# Собираем вакансии по всем ключевым словам
all_vacancies = []
for keyword in keywords:
    all_vacancies.extend(get_all_vacancies(keyword))

# Преобразуем в DataFrame с правильным порядком столбцов
df = pd.DataFrame(all_vacancies, columns=[
    'Название', 'Работодатель', 'Дата публикации', 'Зарплата', 
    'Город', 'Тип занятости', 'Теги вакансии', 
    'Опыт работы', 'Ссылка'
])

# Удаляем дубликаты на основе URL вакансии (столбец "Ссылка")
df = df.drop_duplicates(subset=['Ссылка'])

# Сортировка по названию вакансии и дате публикации
df = df.sort_values(by=['Название', 'Дата публикации'])

# Сохраняем DataFrame в Excel с активными ссылками
filename = 'vacancies.xlsx'

# Создаем новый Excel-файл
with pd.ExcelWriter(filename, engine='openpyxl') as writer:
    df.to_excel(writer, index=False)
    
    # Открываем файл для работы с ссылками
    workbook = writer.book
    worksheet = workbook.active
    
    # Делаем ссылки активными и выводим их как сам адрес
    for row in range(2, len(df) + 2):
        cell = worksheet.cell(row=row, column=9)  # Ссылка находится в 9 колонке
        cell.hyperlink = cell.value  # Добавляем гиперссылку
        cell.font = Font(color="0000FF", underline="single")  # Задаем стиль текста (синий, подчеркнутый)
    
    # Автоматически подгоняем ширину столбцов под содержимое
    for col in worksheet.columns:
        max_length = 0
        column = col[0].column_letter  # Получаем букву столбца
        for cell in col:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2)
        worksheet.column_dimensions[column].width = adjusted_width

print(f"Данные сохранены в файл {filename} с активными ссылками.")


Данные сохранены в файл vacancies.xlsx с активными ссылками.
